In [ ]:
import packages
from configs import settings, const, components
from configs.settings import logger
import asyncio, os, time, yaml, json, datetime, copy, random
from typing import Any, AsyncGenerator, Generator, Callable, Literal, Optional, TypeAlias, Union
from tqdm import tqdm
from pprint import pprint

from toolkit.llm.llama_index import (
	agents, cores, deploys as dpls, evaluation, messages, models, 
	observability, types, utils as utils_llama_index, workflows as wfs
)
from toolkit.llm.llama_index.data import loading, querying, storing

from features.agents.car.tools import VehicleDB
from features.agents.tools import map

from toolkit.utils import utils, typer as t
from toolkit.utils.llm import measure_performance, main as utils_llm
from toolkit.utils.utils import rp_print

In [ ]:
from pymongo import MongoClient
from bson import ObjectId

# Setup

In [ ]:
# MongoDB connection details
MONGO_USERNAME = "root"
MONGO_PASSWORD = "example"
MONGO_HOST = "localhost"
MONGO_PORT = "27017"
MONGO_DB = "car_control_app"

# Construct the MongoDB connection string
mongo_uri = f"mongodb://{MONGO_USERNAME}:{MONGO_PASSWORD}@{MONGO_HOST}:{MONGO_PORT}/?authSource=admin"

# Connect to MongoDB
client = MongoClient(mongo_uri)

# Create or access the database
db = client[MONGO_DB]

# Create or access the collection
vehicles = db['vehicles']


In [ ]:
# Custom JSON encoder to handle ObjectId and datetime
class MongoJSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        if isinstance(o, datetime.datetime):
            return o.isoformat()
        return json.JSONEncoder.default(self, o)

# Function to update a specific field
def update_field(vehicle_id, field_path, new_value):
    result = vehicles.update_one(
        {"vehicleId": vehicle_id},
        {"$set": {field_path: new_value}}
    )
    if result.modified_count > 0:
        print(f"Updated {field_path} for vehicle {vehicle_id}")
    else:
        print(f"No update performed for {field_path}")

# Function to retrieve a specific field
def get_field(vehicle_id, field_path):
    vehicle = vehicles.find_one(
        {"vehicleId": vehicle_id},
        {field_path: 1}
    )
    if vehicle and field_path in vehicle:
        print(f"{field_path} for vehicle {vehicle_id}: {vehicle[field_path]}")
    else:
        print(f"Field {field_path} not found for vehicle {vehicle_id}")

def get_full_document(vehicle_id):
    vehicle = vehicles.find_one({"vehicleId": vehicle_id})
    if vehicle:
        # Convert ObjectId to string for JSON serialization
        vehicle['_id'] = str(vehicle['_id'])
        return json.dumps(vehicle, cls=MongoJSONEncoder, indent=2)
    else:
        return f"Vehicle with ID {vehicle_id} not found"

# Function to add a new alert
def add_alert(vehicle_id, alert_type, message, severity):
    new_alert = {
        "type": alert_type,
        "message": message,
        "severity": severity,
        "timestamp": datetime.datetime.now(),
        "acknowledged": False
    }
    result = vehicles.update_one(
        {"vehicleId": vehicle_id},
        {"$push": {"alerts": new_alert}}
    )
    if result.modified_count > 0:
        print(f"Added new alert for vehicle {vehicle_id}")
    else:
        print(f"Failed to add alert for vehicle {vehicle_id}")

# Function to acknowledge an alert
def acknowledge_alert(vehicle_id, alert_index):
    result = vehicles.update_one(
        {"vehicleId": vehicle_id},
        {"$set": {f"alerts.{alert_index}.acknowledged": True}}
    )
    if result.modified_count > 0:
        print(f"Acknowledged alert at index {alert_index} for vehicle {vehicle_id}")
    else:
        print(f"Failed to acknowledge alert for vehicle {vehicle_id}")

# Sample vehicle document
vehicle_doc = {
    "_id": ObjectId(),
    "vehicleId": "v123",
    "lastUpdated": datetime.datetime.now(),
    
    "vehicleInfo": {
        "basic": {
            "make": "TeslaBMWFord",
            "model": "Model 3X5F-150",
            "year": 2024,
            "vin": "1HGBH41JXMN109186",
            "licensePlate": "ABC1234"
        },
        "status": {
            "powertrain": {
                "engineStatus": "off",
                "batteryLevel": 80,
                "batteryTemperature": 25,
                "range": 300,
                "chargingStatus": "disconnected",
                "fuelLevel": 75,
                "engineTemperature": 90,
                "hybridMode": "auto"
            },
            "driving": {
                "speed": 0,
                "gear": "park",
                "odometer": 15000,
                "tripOdometer": 150.5,
                "parkingBrake": True,
                "tractionControl": True,
                "stabilityControl": True,
                "currentIncline": 0
            }
        }
    },

    "controls": {
        "drivetrain": {
            "steeringMode": "comfort",
            "drivingMode": "normal",
            "regenerativeBraking": "standard",
            "accelerationMode": "standard",
            "suspensionHeight": "normal",
            "startStopSystem": True
        },
        "climate": {
            "temperature": {
                "driver": 22.5,
                "passenger": 22.5,
                "rearLeft": 22.5,
                "rearRight": 22.5
            },
            "fanSpeed": 3,
            "airflow": {
                "face": True,
                "feet": False,
                "windshield": False
            },
            "ac": True,
            "recirculate": False,
            "autoMode": False,
            "seatControls": {
                "driver": {"heat": 0, "ventilation": 0},
                "passenger": {"heat": 0, "ventilation": 0},
                "rearLeft": {"heat": 0},
                "rearRight": {"heat": 0}
            },
            "steeringWheelHeater": False,
            "defrosters": {
                "front": False,
                "rear": False
            },
            "zonedClimate": False,
            "airQuality": {
                "particulateMatter": 10,
                "filtrationLevel": "high"
            },
            "humidityControl": {
                "enabled": False,
                "targetHumidity": 45
            }
        },
        "exterior": {
            "doors": {
                "locked": True,
                "childLock": {
                    "rearLeft": False,
                    "rearRight": False
                },
                "autoLock": True,
                "walkAwayLock": True
            },
            "windows": {
                "driver": 100,
                "passenger": 100,
                "rearLeft": 100,
                "rearRight": 100,
                "childLock": False,
                "antiPinch": True
            },
            "mirrors": {
                "left": {
                    "folded": False,
                    "position": {"vertical": 0, "horizontal": 0},
                    "autoTilt": True,
                    "autoDim": True,
                    "autoTiltReverse": True
                },
                "right": {
                    "folded": False,
                    "position": {"vertical": 0, "horizontal": 0},
                    "autoTilt": True,
                    "autoDim": True,
                    "autoTiltReverse": True
                },
                "rearView": {"autoDim": True}
            },
            "trunk": {"status": "closed", "autoOpen": False},
            "frunk": {"status": "closed"},
            "hood": "closed",
            "sunroof": {
                "position": 0,
                "tilt": 0,
                "autoClose": True
            },
            "chargePort": {
                "status": "closed",
                "lockWhenPlugged": True
            },
            "wipers": {
                "front": {
                    "speed": 0,
                    "on": False,
                    "autoMode": False,
                    "sensitivity": 5,
                    "rainSensorSensitivity": 5
                },
                "rear": {
                    "on": False,
                    "speed": 0
                },
                "washerFluid": {
                    "level": 80,
                    "spray": {"front": False, "rear": False}
                }
            }
        },
        "interior": {
            "steeringWheel": {
                "position": {
                    "tilt": 0,
                    "telescopic": 0
                },
                "easyEntry": True
            },
            "glovebox": "closed",
            "seatPositions": {
                "driver": {"recline": 0, "height": 0, "lumbar": 0},
                "passenger": {"recline": 0, "height": 0, "lumbar": 0},
                "rearLeft": {"recline": 0},
                "rearRight": {"recline": 0}
            },
            "massageFunction": {
                "driver": {"enabled": False, "intensity": 0, "mode": "off"},
                "passenger": {"enabled": False, "intensity": 0, "mode": "off"}
            }
        },
        "lighting": {
            "exterior": {
                "headlights": "auto",
                "foglights": False,
                "daytimeRunningLights": True,
                "highBeamAssist": True,
                "adaptiveHeadlights": True,
                "automaticHighBeam": True
            },
            "interior": {
                "frontDome": "off",
                "rearDome": "off",
                "ambient": {
                    "enabled": True,
                    "color": "#FFFFFF",
                    "brightness": 50
                },
                "readingLights": {
                    "frontLeft": False,
                    "frontRight": False,
                    "rearLeft": False,
                    "rearRight": False
                }
            },
            "welcomeLights": True,
            "leaveHomeLights": {
                "enabled": True,
                "duration": 30
            }
        }
    },

    "assistSystems": {
        "driverAssist": {
            "autopilot": {
                "enabled": False,
                "setSpeed": 70,
                "followDistance": 3
            },
            "parkingAssist": {
                "status": "off",
                "autoParallel": True,
                "autoPerpendicular": True
            },
            "safetyFeatures": {
                "laneAssist": True,
                "emergencyBraking": True,
                "blindSpotWarning": True,
                "speedLimitWarning": True,
                "adaptiveCruiseControl": False,
                "trafficSignRecognition": True,
                "driverAttentionMonitor": True,
                "nightVision": False,
                "crossTrafficAlert": True
            },
            "trafficJamAssist": False,
            "highwayDrivingAssist": False
        },
        "driverProfiles": [
            {
                "name": "Default",
                "settings": {
                    "followDistance": 3,
                    "laneKeepingStrength": "medium"
                }
            }
        ]
    },

    "infotainment": {
        "navigation": {
            "active": False,
            "currentDestination": None,
            "voiceGuidance": {"enabled": True, "volume": 5},
            "alternateRoutes": [],
            "trafficEnabled": True,
            "recentDestinations": [
                {
                    "name": "Home",
                    "address": "123 Main St, Anytown, USA",
                    "coordinates": {"lat": 37.7749, "lng": -122.4194}
                }
            ],
            "savedLocations": [
                {
                    "name": "Work",
                    "address": "456 Office Blvd, Worktown, USA",
                    "coordinates": {"lat": 37.7896, "lng": -122.3978}
                }
            ],
            "preferences": {
                "avoidTolls": False,
                "avoidHighways": False,
                "mapDisplay": "3D"
            },
            "waypoints": [],
            "history": []
        },
        "media": {
            "currentSource": "bluetooth",
            "volume": 6,
            "mute": False,
            "playing": {
                "type": "music",
                "title": "Song Name",
                "artist": "Artist Name",
                "album": "Album Name",
                "duration": 240,
                "currentPosition": 75
            },
            "audioSettings": {
                "bass": 0,
                "treble": 0,
                "midrange": 0,
                "balance": 0,
                "fader": 0,
                "surroundSound": False,
                "equalizer": {
                    "preset": "flat",
                    "custom": [0, 0, 0, 0, 0, 0, 0, 0]
                }
            },
            "audioSources": {
                "bluetooth": True,
                "radio": {
                    "band": "FM",
                    "frequency": 98.5
                },
                "aux": False,
                "streaming": {
                    "active": False,
                    "service": None
                }
            },
            "podcast": {
                "currentEpisode": None,
                "playbackSpeed": 1
            },
            "audiobook": {
                "currentBook": None,
                "currentChapter": None,
                "playbackSpeed": 1
            }
        },
        "phone": {
            "connected": True,
            "deviceName": "John's iPhone",
            "batteryLevel": 75,
            "signal": 4,
            "recentCalls": [
                {
                    "contactName": "Jane Doe",
                    "phoneNumber": "+1234567890",
                    "timestamp": datetime.datetime.now(),
                    "type": "outgoing"
                }
            ],
            "contacts": [
                {
                    "name": "Jane Doe",
                    "phoneNumber": "+1234567890",
                    "favorite": True
                }
            ],
            "bluetooth": {
                "pairedDevices": ["John's iPhone", "Mary's Android"],
                "autoConnect": True
            },
            "doNotDisturb": False,
            "callForwarding": {
                "enabled": False,
                "number": None
            },
            "voicemail": {
                "accessNumber": None,
                "unreadMessages": 0
            }
        },
        "speechRecognition": {
            "active": False,
            "wakePhrases": ["Hey Car", "OK Car"]
        }
    },

    "displaySettings": {
        "mainDisplay": {
            "brightness": 80,
            "nightMode": False,
            "theme": "light",
            "screenLayout": "standard",
            "widgetLayout": [
                {"type": "media", "position": "topLeft"},
                {"type": "navigation", "position": "center"}
            ]
        },
        "headUpDisplay": {
            "enabled": True,
            "brightness": 70,
            "content": ["speed", "navigation", "alerts"]
        },
        "driverInformationDisplay": {
            "layout": "standard",
            "customFields": ["range", "tirePressure", "outsideTemp"]
        },
        "units": {
            "temperature": "celsius",
            "distance": "kilometers",
            "time": "24h",
            "pressure": "psi"
        },
        "language": "en-US"
    },

    "security": {
        "alarm": {
            "enabled": True,
            "triggered": False,
            "sensitivity": "medium"
        },
        "pinToDisable": False,
        "recentActivity": [
            {
                "event": "door_unlock",
                "timestamp": datetime.datetime.now(),
                "method": "key_fob"
            }
        ],
        "valetMode": False,
        "dashcam": {
            "active": True,
            "storage": 75
        },
        "sentryMode": {
            "enabled": True,
            "excludeHome": True,
            "excludeWork": False
        },
        "biometricAccess": {
            "fingerprint": {
                "enabled": False,
                "registeredUsers": []
            },
            "facialRecognition": {
                "enabled": False,
                "registeredUsers": []
            }
        },
        "geofencing": {
            "enabled": False,
            "boundaries": []
        }
    },

    "maintenance": {
        "oilLife": 80,
        "brakeWear": {
            "frontLeft": 90,
            "frontRight": 90,
            "rearLeft": 95,
            "rearRight": 95
        },
        "nextService": datetime.datetime(2025, 4, 2),
        "fluidLevels": {
            "washerFluid": 80,
            "brakeFluid": 90,
            "coolant": 85
        },
        "batteryHealth": 95,
        "suspensionSystem": {
            "frontLeft": "ok",
            "frontRight": "ok",
            "rearLeft": "ok",
            "rearRight": "ok"
        },
        "tirePressure": {
            "frontLeft": 32,
            "frontRight": 32,
            "rearLeft": 32,
            "rearRight": 32,
            "spare": 35
        }
    },

    "chargingSettings": {
        "chargeLimit": 90,
        "scheduledCharging": {
            "enabled": False,
            "startTime": "22:00",
            "endTime": "06:00"
        },
        "fastChargingEnabled": True,
        "preconditioning": False,
        "preferredChargingLocations": [
            {"name": "Home", "location": {"lat": 37.7749, "lng": -122.4194}},
            {"name": "Work", "location": {"lat": 37.7896, "lng": -122.3978}}
        ],
        "smartGridIntegration": {
            "enabled": False,
            "preferredChargingTimes": [
                {"start": "01:00", "end": "05:00"}
            ]
        }
    },

    "alerts": [
        {
            "type": "maintenance",
            "message": "Tire rotation recommended",
            "severity": "low",
            "timestamp": datetime.datetime.now(),
            "acknowledged": False
        }
    ]
}

# Convert the document to a JSON-serializable format
# vehicle_doc_serializable = json.loads(json.dumps(vehicle_doc, cls=MongoJSONEncoder))

# Insert the vehicle document into the collection
# result = vehicles.insert_one(vehicle_doc)
# print(f"Inserted vehicle with id: {result.inserted_id}")

# Create indexes for faster querying
# vehicles.create_index("vehicleId", unique=True)
# vehicles.create_index("vehicleInfo.basic.vin", unique=True)

# # Example usage of update_field function
# update_field("v123", "controls.climate.temperature.driver", 23.5)

# # Example usage of get_field function
# get_field("v123", "controls.climate.temperature.driver")

# # Example usage of add_alert function
# add_alert("v123", "low_fuel", "Fuel level below 15%", "medium")

# # Example usage of acknowledge_alert function
# acknowledge_alert("v123", 0)

# print(get_full_document("v123"))

In [ ]:
# # Close the MongoDB connection
# client.close()
# print("MongoDB connection closed")

# Agent

## CarOperator

In [ ]:
tools_control = agents.add_tools(
	[agents.FunctionTool.from_defaults(async_fn=VehicleDB.db_mongo_vehicle.process_user_query)],
)

In [ ]:
agent_control = agents.MyAgent(
	llm=settings.llm_control,
	use_local_model=settings.use_local_model,
	tools=tools_control,
	system_prompt=settings.prompts_agent_car["control"]["System_VehicleDB"]["dev"],
	verbose=True,
	name="Control Agent",
).agent

# agent = agents.ReActAgent.from_tools(
#   tools=tools, 
# 	llm=llm_control,
# 	verbose=True,
# 	# output_parser=agents.ReActOutputParser(),
# 	context=prompts_agent_car["control"]["System_VehicleDB"]["dev"], # System prompt
# )


In [ ]:
queries = [
	"What is current fan speed",
	"Set fan speed to 4",
	"What is current driver temperature",
	"Increase driver temperature",
	"Set driver temperature to 20",
	"What is the current status of the AC",
	"Activate the AC mode",
	"What is current front wiper speed",
	"Increase front wiper speed",
	"Help me Unlock doors",
	"What is VIN of this car",
	
	"Fold the mirrors",
	"Unfold the mirrors",
	"Direct air to feet only",
	"Direct air to face and feet",
	
	"Is the car locked",
	"Is the car trunk opened?",
	"Open the car trunk",
 
	"It's too hot",
	"I'm feeling sleepy",
	"The radio is too loud",
	"Can you turn on the AC?",
	"The windshield is getting foggy",
	"The headlights seem dim",
]

In [ ]:
result = await utils_llama_index.interact_agent(
	agent=agent_control,
	# user_query=queries[0],
	user_query="Yes",
	# user_query="It's too hot",
	# user_query="Activate the AC mode. Increase front wiper speed. Help me Unlock doors",
	mode="achat",
)

rp_print(result)

## InfoNavigator

In [ ]:
# pip install openmeteo-requests
# pip install openmeteo-sdk
# pip install retry-requests
# pip install requests-cache
# pip install polars

## General

In [ ]:
tools_general = agents.add_tools(
	[agents.FunctionTool.from_defaults(fn=tool_weather.process_weather_query)],
)

agent_general = agents.MyAgent(
	llm=settings.llm_general,
	use_local_model=settings.use_local_model,
	tools=tools_general,
	system_prompt=settings.prompts_agent_car["general"]["System_General"]["dev"],
	verbose=True,
	name="General Agent",
).agent

In [ ]:
result = await utils_llama_index.interact_agent(
	agent=agent_general,
	# user_query=queries[0],
	user_query="What is the current temperature?",
	# user_query="What's my current location?",
	mode="achat",
)

rp_print(result)

## Navigation

In [ ]:
from features.agents.tools import map


tools_map = agents.add_tools(
	[agents.FunctionTool.from_defaults(async_fn=map.process_user_query)],
)

agent_map = agents.MyAgent(
	llm=settings.llm_map,
	use_local_model=settings.use_local_model,
	tools=tools_map,
	system_prompt=settings.prompts_agent_car["map"]["System"]["dev"],
	verbose=True,
	name="Map Agent",
).agent

In [ ]:
result = await utils_llama_index.interact_agent(
	agent=agent_map,
	# user_query="Tìm quán cháo vịt quanh đây",
	user_query="Tìm quán trà sữa quanh đây",
)
pprint(result)

In [ ]:
result = await utils_llama_index.interact_agent(
	agent=agent_map,
	user_query="Chỉ đường đến Quán Trà Sữa Apa",
	# user_query="Chỉ đường đến Quán cháo vịt Thanh đa",
	# user_query="Chỉ đường đến Quán cháo vịt Lái thiêu",
)
pprint(result)

# Dev

In [ ]:
user_query = "Could you provide some tips or notes to consider before driving a car?"

prompt_categorize_query = prompts_agent_car["CategorizeQuestion"]["dev"]

query_category = await utils_llama_index.interact_model(
  prompt=prompt_categorize_query, mode="achat", user_query=user_query,
)

In [ ]:
token_generator = utils_llama_index.interact_model(
  prompt=prompt_categorize_query, mode="astream", user_query=user_query,
  measure_performance=True,
)

async for token in await token_generator:
  print(token, end="", flush=True)

---

In [ ]:
await utils.time_it(
	lambda: VehicleDB.db_mongo_vehicle.get_full_document("v123"),
	name="[MongoDB] Get full data",
	n_times=10,
)

In [ ]:
await utils.time_it(
	lambda: VehicleDB.db_mongo_vehicle.get_field(
		"v123", field_path="controls.climate.ac"
	),
	name="[MongoDB] Get data",
	# n_times=10,
)

In [ ]:
await utils.time_it(
	lambda: VehicleDB.db_mongo_vehicle.update_field(
		"v123", field_path="controls.climate.ac", new_value=False,
	),
	name="[MongoDB] Update data",
	# n_times=10,
)

In [ ]:
await utils.time_it(
	lambda: VehicleDB.db_mongo_vehicle.parse_user_query(
		"Open the trunk"
	),
	name="[MongoDB + LLM] Parse User Query",
	# n_times=10,
)

In [ ]:
VehicleDB.db_mongo_vehicle.parse_user_query(
	"Is the trunk open"
)